# Calculating the number of households within certain distances from tanks for each county using ```.sjoin()```

### Import statements

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
import geopandas as gpd
import dask
from dask import dataframe as dd
import dask_geopandas

### Reading county shapefile dataframe
To count the number of households within five miles of a storage tank for each county, we use GeoPandas' ```.sjoin()``` method. Using this method, we will perform a spatial join between each county's geometry and the dataframe including Point geometries for each household in the US. For this, we need a dataframe with geometries for all counties in the US- which we took from the United States Census Bureau's Cartographic Boundary Files (available [here](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)). Then, we filter to exclude counties from Alaska, Hawaii, Puerto Rico, Virgin Islands, American Samoa, Guam, Northern Marian Islands, as there are no tanks in those regions in the AST dataset. We also drop unnecessary columns.

In [4]:
df_counties = gpd.read_file('/hpc/group/codeplus22-vis/county_shp_files/us_counties.shp')
df_counties = df_counties[df_counties['STATEFP'] != '02']
df_counties = df_counties[df_counties['STATEFP'] != '15']
df_counties = df_counties[df_counties['STATEFP'] != '72']
df_counties = df_counties[df_counties['STATEFP'] != '78']
df_counties = df_counties[df_counties['STATEFP'] != '60']
df_counties = df_counties[df_counties['STATEFP'] != '66']
df_counties = df_counties[df_counties['STATEFP'] != '69']
df_counties = df_counties[['NAME', 'geometry']]
df_counties.rename(columns = {'NAME': 'county'}, inplace = True)
df_counties

,county,geometry
0,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...
3228,Gosper,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3229,Holmes,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3230,Gillespie,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3231,Milwaukee,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


### Reading InfoUSA Data (pre-processed)
This InfoUSA data was preprocessed to include the distance between each household and the nearest tank. We need this information to filter for only households within five miles of a storage tank, which we do below.

In [8]:
df_hh = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/distances_all_hh_with_children_final.parquet')
df_hh.head()

,zip,county,state,child_num,age_code,lat_h_3857,lon_h_3857,lat_h_4326,lon_h_4326,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,distance_m,distance_mi,distance_category
0,18833,15,PA,1,C,-8.509454e+06,5.101307e+06,41.600392,-76.441724,2.050670,15.375901,5.380037,14.512438,-1.0,17.062917,9.063660,53847.632898,33.459368,4
1,18833,15,PA,1,H,-8.499018e+06,5.096218e+06,41.566196,-76.347977,4.881886,15.876431,4.895073,24.892845,-1.0,30.218719,13.460825,45869.438119,28.501947,4
2,18833,15,PA,0,E,-8.496356e+06,5.099448e+06,41.587904,-76.324061,4.881886,15.876431,4.895073,24.892845,-1.0,30.218719,13.460825,46015.805516,28.592896,4
3,18833,15,PA,1,G,-8.509963e+06,5.103102e+06,41.612450,-76.446301,2.050670,15.375901,5.380037,14.512438,-1.0,17.062917,9.063660,54518.419780,33.876175,4
4,18833,15,PA,1,G,-8.508370e+06,5.099066e+06,41.585339,-76.431989,2.050670,15.375901,5.380037,14.512438,-1.0,17.062917,9.063660,53297.730315,33.117674,4


We also only keep the ```lat_h_4326``` and ```lon_h_4326``` columns because these are the coordinates for each household we will use to create the Point geometries for each household in the households GeoDataFrame. We use the coordinates in EPSG 4326 rather than the coordinates in EPSG 3857 because the geometries in the ```df_counties``` GeoDataFrame are in EPSG 4326. The coordinate system must remain consistent in order to get accurate results when performing the spatial join with ```.sjoin()```.

In [9]:
df_hh = df_hh[['lat_h_4326', 'lon_h_4326', 'distance_category']]
df_hh = df_hh[df_hh['distance_category'] != 4]
df_hh.head()

,lat_h_4326,lon_h_4326,distance_category
2976,43.0871,-77.5982,3
2977,43.0871,-77.5982,3
2978,43.0871,-77.5982,3
2979,43.0871,-77.5982,3
2980,43.0871,-77.5982,3


### Use Dask to transform pandas dataframe to a geopandas dataframe
To perform a spatial join between two dataframes, each of these dataframes must be GeoDataFrames- thus, we must convert our ```df_hh``` dataframe to a GeoDataFrame. However, as this dataframe has more than 17 million rows, converting it without using Dask is not time-efficient. Hence, we turned to Dask, an open-source Python library for parallel computing. It allows us to efficiently execute the transformation of our dataframe to a GeoDataFrame, even when working with over 17 million rows. 

To use Dask, we first converted our dataframe to a Dask dataframe, using Dask's ```.from_pandas()``` method. This method takes in our pandas dataframe along with the ```npartitions``` parameter, which is used to specify the number of 'sections' the dask dataframe will be split into.

In [10]:
df_dask = dd.from_pandas(df_hh, npartitions = 500)

Then, we specify what manipulation to the dask dataframe ```df_dask``` to compute. In this case, we use Dask Geopandas' ```.points_from_xy()``` method to convert the pandas dask dataframe into a geopandas dask dataframe.

In [11]:
%%time
df_dask['geometry'] = dask_geopandas.points_from_xy(df_dask, 'lon_h_4326', 'lat_h_4326')

CPU times: user 7.7 s, sys: 1.08 s, total: 8.77 s
Wall time: 8.79 s


After, we convert the dask geodataframe into a geopandas dataframe:

In [12]:
%%time
gdf = dask_geopandas.from_dask_dataframe(df_dask)

CPU times: user 6.67 ms, sys: 230 µs, total: 6.9 ms
Wall time: 6.62 ms


Calling compute puts all the above code into action. Dask executes each set of commands on each partition, as specified above. This returns GeoDataFrame ```gdf_hh```, with over 17 million rows, in around five seconds.

In [13]:
%%time
gdf_hh = gdf.compute()

CPU times: user 7.07 s, sys: 4.54 s, total: 11.6 s
Wall time: 5.45 s


In [14]:
gdf_hh = gdf_hh[['distance_category', 'geometry']]
gdf_hh.head()

,distance_category,geometry
2976,3,POINT (-77.59820 43.08710)
2977,3,POINT (-77.59820 43.08710)
2978,3,POINT (-77.59820 43.08710)
2979,3,POINT (-77.59820 43.08710)
2980,3,POINT (-77.59820 43.08710)


### Computing number of households within 0.5 mi of a storage tank in each county
Now that we have our two GeoDataFrames, we ```.sjoin()``` to find which households are within 0.5 miles from a tank in each county.

#### Filtering the household data for only households within 0.5 mi of a storage tank
First, we created a new GeoDataFrame ```gdf_hh_half_mi``` that included only the households within 0.5 miles of a storage tank.

In [15]:
gdf_hh_half_mi = gdf_hh[gdf_hh['distance_category'] == 1]
gdf_hh_half_mi.head()

,distance_category,geometry
3348,1,POINT (-73.66289 40.99470)
3354,1,POINT (-73.66757 41.00379)
3355,1,POINT (-73.66548 40.99359)
3356,1,POINT (-73.66699 41.00108)
3365,1,POINT (-73.67412 41.00968)


#### Adding column in county dataframe with the number of households within 0.5mi of a tank
Then, we used ```.sjoin()``` to find the number of households within 0.5 miles of a tank for each county.

Calling ```.sjoin()``` and passing in two GeoDataFrames will return a new GeoDataFrame that only includes the observations with geometries that are the intersections of the two original GeoDataFrames. In our case, passing in a GeoDataFrame with the geometry for Harris County and a GeoDataFrame with all the households within 0.5 miles of a tank to the ```.sjoin()``` method returns a new GeoDataFrame with all the households within 0.5 miles of a tank in Harris County, as it returns all the Point geometries that intersects the Harris county Polygon geometry. Computing the length of that new dataframe through ```len()``` returns the number of households within 0.5 miles in Harris county, then.

However, we need to do this for all counties in the US, so we use for loop. This loop iterates every row of ```df_counties``` finds the intersection between that county and the households within 0.5 miles of a tank (```gdf_hh_half_mi```), computes the length of that ```intersect_df```, and adds that number to a new column in ```df_counties```. We add the column to ```df_counties``` specifically, because then we can use this exact dataframe to plot a map of the US where each county is colored by the number of households within five miles of a storage tank, as it contains all the geometries for each county required for a GeoViews visualization.

In [16]:
%%time

df_counties['hh_half_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_half_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    # print(num)
    
    # adding num to df_counties column
    df_counties['hh_half_mi'].iloc[i] = num

df_counties

CPU times: user 13min 37s, sys: 1min 59s, total: 15min 37s
Wall time: 15min 42s


,county,geometry,hh_half_mi
0,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",0
1,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",0
2,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",0
3,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",0
4,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",0
...,...,...,...
3228,Gosper,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",0
3229,Holmes,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",0
3230,Gillespie,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",0
3231,Milwaukee,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",1595


In [17]:
df_counties = gpd.read_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all_final.shp')
df_counties

,county,hh_half_mi,geometry
0,Ballard,0,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,Bourbon,0,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,Butler,0,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,Estill,0,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,Fleming,0,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...
3103,Gosper,0,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3104,Holmes,0,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3105,Gillespie,0,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3106,Milwaukee,1595,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


### Computing number of households within 1 mi of a storage tank in each county
Following the same steps as described above, except filtering for households within one mile of a storage tank.

#### Filtering the household data for only households within 1 mi of a storage tank

In [15]:
gdf_hh_one_mi = gdf_hh[gdf_hh['distance_category'] == 2]
gdf_hh_one_mi

,distance_category,geometry
3336,2,POINT (-73.66740 41.01586)
3338,2,POINT (-73.66183 41.01394)
3339,2,POINT (-73.65955 41.00575)
3344,2,POINT (-73.68580 41.00382)
3346,2,POINT (-73.68269 41.01059)
...,...,...
53059066,2,POINT (-98.45835 33.93817)
53059067,2,POINT (-98.41700 33.91695)
53059079,2,POINT (-98.50385 33.94957)
53059082,2,POINT (-98.40050 33.91524)


#### Adding column in county dataframe with the number of households within 1 mi of a tank
Takes around 30min

In [16]:
%%time

df_counties['hh_one_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_one_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    # print(num)
    
    # adding num to df_counties column
    df_counties['hh_one_mi'].iloc[i] = num

df_counties


KeyboardInterrupt



In [20]:
df_counties.to_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all_final.shp')

In [3]:
df_counties = gpd.read_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all_final.shp')
df_counties

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,HH_half_mi,HH_one_mi,HH_five_mi,HH_total,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,0,0,0,0,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,0,0,0,0,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,0,0,0,0,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,0,0,0,0,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,0,0,0,0,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,0,0,0,0,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3104,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,0,0,0,0,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3105,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,0,0,0,0,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3106,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,1595,8269,106533,116397,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


### Computing number of households within 5mi of a storage tank in each county
Following the same steps as described above, except filtering for households within five miles of a storage tank. Because the households GeoDataFrame has over 14 million rows, this process takes quite a while.

#### Filtering the household data for only households within 5 mi of a storage tank

In [18]:
gdf_hh_five_mi = gdf_hh[gdf_hh['distance_category'] == 3]
gdf_hh_five_mi

,distance_category,geometry
2976,3,POINT (-77.59820 43.08710)
2977,3,POINT (-77.59820 43.08710)
2978,3,POINT (-77.59820 43.08710)
2979,3,POINT (-77.59820 43.08710)
2980,3,POINT (-77.59820 43.08710)
...,...,...
53059098,3,POINT (-98.58665 33.92580)
53059100,3,POINT (-98.57742 33.91278)
53059101,3,POINT (-98.33038 33.91823)
53059103,3,POINT (-98.41964 33.89710)


#### Adding column in county dataframe with the number of households within 5km of a tank
Takes around 4 hours.

In [ ]:
%%time 

df_counties['hh_five_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_five_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    
    # adding num to df_counties column 
    df_counties['hh_five_mi'].iloc[i] = num

df_counties

In [13]:
df_counties['HH_total'] = df_counties['HH_half_mi'] + df_counties['HH_one_mi'] + df_counties['HH_five_mi']
df_counties

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,HH_half_mi,HH_one_mi,geometry,HH_five_mi,HH_total
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,0,0,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",0,0
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,0,0,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",0,0
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,0,0,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",0,0
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,0,0,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",0,0
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,0,0,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,0,0,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",0,0
3104,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,0,0,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",0,0
3105,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,0,0,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",0,0
3106,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,1595,8269,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",106533,116397


In [14]:
df_counties.to_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all_final.shp')

In [16]:
# df_counties = df_counties[['STATEFP', 'COUNTYFP', 'NAME', 'LSAD', 'HH_field_only', 'HH_mi_only', 'HH_km_only', 'HH_km',
#                            'geometry']]
# df_counties.rename(columns = {'HH_field_only': 'HH_field', 'HH_mi_only': 'HH_mi', 'HH_km_only': 'HH_km', 
#                              'HH_km': 'HH_total'}, inplace = True)
# df_counties

,STATEFP,COUNTYFP,NAME,LSAD,HH_field,HH_mi,HH_km,geometry
0,21,007,Ballard,06,0,0,0,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,Bourbon,06,0,0,0,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,Butler,06,0,0,0,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,Estill,06,0,0,0,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,Fleming,06,0,0,0,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...,...,...,...,...,...
3103,31,073,Gosper,06,0,0,0,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3104,39,075,Holmes,06,0,0,0,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3105,48,171,Gillespie,06,0,0,0,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3106,55,079,Milwaukee,06,1,9864,63587,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


### Exporting it as a shapefile

In [ ]:
df_counties.to_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all.shp')

In [15]:
df = gpd.read_file('/hpc/group/codeplus22-vis/infousa_copy/hh_num_counties_all.shp')
df

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,HH_half_mi,HH_one_mi,HH_five_mi,HH_total,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,0,0,0,0,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,0,0,0,0,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,0,0,0,0,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,0,0,0,0,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,0,0,0,0,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,0,0,0,0,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3104,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,0,0,0,0,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3105,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,0,0,0,0,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3106,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,1595,8269,106533,116397,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."
